In [ ]:
import numpy as np
import pandas as pd
import joblib
from collections import Counter

# Load the entire pipeline (preprocessor + model)
pipeline = joblib.load("../../web-app/hemant-gulati/pipeline.pkl")
print("Pipeline components:", pipeline.steps)

print("Pipeline type:", type(pipeline))  # Should show sklearn Pipeline

# Collect user inputs
hbA1c_level = float(input("Enter HbA1c Level (e.g., 5.5): "))
blood_glucose_level = int(input("Enter Blood Glucose Level (e.g., 100): "))
bmi = float(input("Enter BMI (e.g., 24.5): "))
age = int(input("Enter Age (e.g., 45): "))
hypertension = int(input("Enter Hypertension (0 for No, 1 for Yes): "))
smoking_history = int(input("Enter Smoking History (0 for No, 1 for Yes): "))
heart_disease = int(input("Enter Heart Disease (0 for No, 1 for Yes): "))

# Prepare user data in a dictionary format
user_data = {
    'hbA1c_level': [hbA1c_level],
    'blood_glucose_level': [blood_glucose_level],
    'bmi': [bmi],
    'age': [age],
    'hypertension': [hypertension],
    'smoking_history': [smoking_history],
    'heart_disease': [heart_disease]
}

# Convert user input to DataFrame
user_df = pd.DataFrame(user_data)

# Automatically generate binned columns
user_df['binned_blood_glucose_Diabetes'] = (user_df['blood_glucose_level'] > 125).astype(int)
user_df['binned_hba1c_Severe Diabetes'] = (user_df['hbA1c_level'] > 7.5).astype(int)
user_df['binned_bmi_Obese'] = (user_df['bmi'] >= 30).astype(int)

# Define the required feature columns
top_features = [
    'hbA1c_level', 'blood_glucose_level', 'bmi', 'age', 
    'hypertension', 'smoking_history', 'heart_disease',
    'binned_blood_glucose_Diabetes', 'binned_hba1c_Severe Diabetes', 'binned_bmi_Obese'
]

# Ensure the DataFrame has all required columns
user_df_final = user_df[top_features]

# Make prediction with the loaded pipeline
prediction = pipeline.predict(user_df_final)[0]
prediction_proba = pipeline.predict_proba(user_df_final)[0]

# Interpret the prediction
if prediction == 0:
    prediction_label = "Non-diabetic"
elif prediction == 1:
    prediction_label = "Pre-diabetic"
else:
    prediction_label = "Diabetic"

# Display the prediction and confidence level
print(f"Prediction: {prediction_label}")
print(f"Confidence Level: {np.max(prediction_proba) * 100:.2f}%")


Pipeline components: [('preprocessor', ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['hbA1c_level', 'blood_glucose_level', 'bmi',
                                  'age']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['binned_blood_glucose_Diabetes',
                                  'binned_hba1c_Severe Diabetes',
                                  'hypertension', 'smoking_history',
                                  'heart_disease', 'binned_bmi_Obese'])])), ('model', GradientBoostingClassifier(max_depth=5, random_state=42))]
Pipeline type: <class 'sklearn.pipeline.Pipeline'>
